In [ ]:
import pandas as pd
import numpy as np
import glob
import os

# Time encoding function
def encode_time(value, max_value):
    value_sin = np.sin(2 * np.pi * value / max_value)
    value_cos = np.cos(2 * np.pi * value / max_value)
    return value_sin, value_cos

# Function to process marginal data
def process_marginal(file_path):
    print(f"Processing marginal file: {file_path}")
    try:
        data = pd.read_csv(file_path)
        data = data.rename(columns={'DA ES': 'DA'})
        data = data[['Year', 'Month', 'Day', 'Hour', 'DA']].dropna()
        return data
    except Exception as e:
        print(f"Error processing marginal file {file_path}: {e}")
        return None

# Function to process intra-day data
def process_intra_day(file_path):
    print(f"Processing intra-day file: {file_path}")
    try:
        data = pd.read_csv(file_path)
        data = data.rename(columns={'AvgES': 'ID'})
        data = data[['Year', 'Month', 'Day', 'Hour', 'ID']].dropna()
        return data
    except Exception as e:
        print(f"Error processing intra-day file {file_path}: {e}")
        return None


# Function to process gas price data
def process_gas(file_paths):
    print("Processing gas price files...")
    all_gas_data = []
    for file_path in file_paths:
        try:
            # Load the Excel file
            gas_data = pd.read_excel(file_path, sheet_name="Regulated gas")
            
            # Use column positions to rename, assuming consistent format
            gas_data = gas_data.rename(columns={
                gas_data.columns[0]: 'Trading_Day',
                gas_data.columns[4]: 'Price_EUR_MWh',
                gas_data.columns[5]: 'Volume_MWh'
            })
            
            # Convert 'Trading_Day' to datetime
            gas_data['Trading_Day'] = pd.to_datetime(gas_data['Trading_Day'], errors='coerce')
            
            # Drop rows with invalid dates
            gas_data = gas_data.dropna(subset=['Trading_Day'])
            
            # Extract Year, Month, and Day for merging
            gas_data['Year'] = gas_data['Trading_Day'].dt.year
            gas_data['Month'] = gas_data['Trading_Day'].dt.month
            gas_data['Day'] = gas_data['Trading_Day'].dt.day
            
            # Sort the data to ensure proper forward-fill order
            gas_data = gas_data.sort_values(by=['Year', 'Month', 'Day'])
            
            # Fill missing prices and volumes using forward-fill
            gas_data['Price_EUR_MWh'] = gas_data['Price_EUR_MWh'].fillna(method='ffill')
            gas_data['Volume_MWh'] = gas_data['Volume_MWh'].fillna(method='ffill')
            
            # Keep only relevant columns
            gas_data = gas_data[['Year', 'Month', 'Day', 'Price_EUR_MWh', 'Volume_MWh']]
            
            # Append to the list
            all_gas_data.append(gas_data)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    
    # Combine all gas data
    combined_gas_data = pd.concat(all_gas_data, ignore_index=True)
    
    # Remove duplicates
    combined_gas_data = combined_gas_data.drop_duplicates(subset=['Year', 'Month', 'Day'])
    
    print("Gas data processed successfully.")
    return combined_gas_data

In [33]:
import pandas as pd
import numpy as np
import glob
import os

# Define directories
data_dir = '../raw_data/'
output_dir = '../formatted_data/'

# Discover files
marginal_files = glob.glob(os.path.join(data_dir, 'data_electric/data_da*.csv'))
intra_day_files = glob.glob(os.path.join(data_dir, 'data_electric/data_id*.csv'))
gas_files = glob.glob(os.path.join(data_dir, 'data_gas/original_gas/*.xlsx'))

print(f"Found {len(marginal_files)} marginal files")
print(f"Found {len(intra_day_files)} intra-day files")
print(f"Found {len(gas_files)} gas price files")

# Time encoding function
def encode_time(value, max_value):
    value_sin = np.sin(2 * np.pi * value / max_value)
    value_cos = np.cos(2 * np.pi * value / max_value)
    return value_sin, value_cos

# Process marginal and intra-day data
marginal_data = pd.concat(
    [df for file in marginal_files if (df := process_marginal(file)) is not None and not df.empty],
    ignore_index=True
)
intra_day_data = pd.concat(
    [df for file in intra_day_files if (df := process_intra_day(file)) is not None and not df.empty],
    ignore_index=True
)

# Process gas data from XLSX files
gas_data = process_gas(gas_files)

# Merge marginal and intra-day data
final_data = pd.merge(marginal_data, intra_day_data, on=['Year', 'Month', 'Day', 'Hour'], how='inner')

# Optionally merge gas price data
final_data = pd.merge(final_data, gas_data, on=['Year', 'Month', 'Day'], how='left')

# Add encoded time columns
final_data['Hour_Sin'], final_data['Hour_Cos'] = zip(*final_data['Hour'].apply(lambda x: encode_time(x, 24)))
final_data['Day_Sin'], final_data['Day_Cos'] = zip(*final_data['Day'].apply(lambda x: encode_time(x, 31)))
final_data['Month_Sin'], final_data['Month_Cos'] = zip(*final_data['Month'].apply(lambda x: encode_time(x, 12)))

# Add scaled Year and price difference
final_data['Year_Scaled'] = (final_data['Year'] - 2018) * 0.1 + 0.1
final_data['Diff'] = final_data['DA'] - final_data['ID']

# Save the processed data
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'formatted_data.csv')
final_data.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")

# Validation
missing_values = final_data.isnull().sum()
print("Missing values:")
print(missing_values)

duplicate_dates = final_data.duplicated(subset=['Year', 'Month', 'Day', 'Hour'])
print("Number of duplicate dates:", duplicate_dates.sum())


Found 2 marginal files
Found 2 intra-day files
Found 7 gas price files
Processing marginal file: ../raw_data/data_electric/data_da_23.csv
Processing marginal file: ../raw_data/data_electric/data_da_24.csv
Processing intra-day file: ../raw_data/data_electric/data_id_24.csv
Processing intra-day file: ../raw_data/data_electric/data_id_23.csv
Processing gas price files...


/tmp/ipykernel_779085/840273746.py:71: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gas_data['Price_EUR_MWh'] = gas_data['Price_EUR_MWh'].fillna(method='ffill')
/tmp/ipykernel_779085/840273746.py:72: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gas_data['Volume_MWh'] = gas_data['Volume_MWh'].fillna(method='ffill')
/tmp/ipykernel_779085/840273746.py:71: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gas_data['Price_EUR_MWh'] = gas_data['Price_EUR_MWh'].fillna(method='ffill')
/tmp/ipykernel_779085/840273746.py:72: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gas_data['Volume_MWh'] = gas_data['Volume_MWh'].fillna(method='ffill')
/tmp/ipy

Gas data processed successfully.
Processed data saved to ../formatted_data/formatted_data.csv
Missing values:
Year                0
Month               0
Day                 0
Hour                0
DA                  0
ID                  0
Price_EUR_MWh    6527
Volume_MWh       6527
Hour_Sin            0
Hour_Cos            0
Day_Sin             0
Day_Cos             0
Month_Sin           0
Month_Cos           0
Year_Scaled         0
Diff                0
dtype: int64
Number of duplicate dates: 0


In [34]:
import pandas as pd

# Load the final file
final_data = pd.read_csv('../formatted_data/formatted_data.csv')

# Check for missing values in `Price_EUR_MWh` and `Volume_MWh`
missing_price = final_data[final_data['Price_EUR_MWh'].isnull()]
missing_volume = final_data[final_data['Volume_MWh'].isnull()]

# Split missing data by year and month
missing_price_by_year_month = missing_price.groupby(['Year', 'Month']).size().reset_index(name='Missing_Price_Count')
missing_volume_by_year_month = missing_volume.groupby(['Year', 'Month']).size().reset_index(name='Missing_Volume_Count')

# Merge the results for comparison
missing_summary = pd.merge(
    missing_price_by_year_month,
    missing_volume_by_year_month,
    on=['Year', 'Month'],
    how='outer'
).fillna(0)  # Fill NaN with 0 for missing counts

# Save results to CSV for reference
missing_summary.to_csv('../formatted_data/missing_summary.csv', index=False)

# Print results
print("Summary of missing values by Year and Month:")
print(missing_summary)


Summary of missing values by Year and Month:
      Year  Month  Missing_Price_Count  Missing_Volume_Count
0   2023.0    1.0                  168                   168
1   2023.0    2.0                   24                    24
2   2023.0    3.0                   96                    96
3   2023.0    4.0                   24                    24
4   2023.0    5.0                   48                    48
5   2023.0    6.0                  120                   120
6   2023.0    7.0                  720                   720
7   2023.0    8.0                  168                   168
8   2023.0    9.0                  144                   144
9   2023.0   10.0                  240                   240
10  2023.0   11.0                  288                   288
11  2023.0   12.0                  336                   336
12  2024.0    1.0                  264                   264
13  2024.0    2.0                  216                   216
14  2024.0    3.0                  383  